In [1]:
import fitz,time,re,os
from PyPDF4 import PdfFileReader
from docx import  Document
from docx.shared import Inches

In [2]:
# 提取图片
def pdf2pic(path,pic_path,document):
    '''
    # 从指定pdf中提取图片并保存到指定路径
    :param path: pdf路径
    :param pic_path: 图片保存路径 
    '''
    # 使用正则寻找图片
    checkXO = r"/Type(?= */XObject)" 
    findIM=r"/Subtype(?= */Image)"
    pdf=fitz.open(path)
    imgcount=0
    lenXREF=pdf.xref_length()
    for i in range(1,lenXREF):
        # 定义对象字符串
        text = pdf.xref_object(i)
        isXObject = re.search(checkXO, text)
        # 使用正则表达式查看是否是图片
        isImage = re.search(findIM, text)
        # 如果不是对象也不是图片，则continue
        if not isXObject or not isImage:
            continue
        imgcount +=1
        # 根据索引生成图像
        pic = fitz.Pixmap(pdf, i)
        # 根据pdf的路径生成图片的名称
        new_name = "img{}.png".format(imgcount)
        # 如果pix.n<5,可以直接存为PNG
        if pic.n < 3:
            pic.save(os.path.join(pic_path, new_name))
            document.add_picture(os.path.join(pic_path, new_name),width=Inches(5.5))
        # 否则先转换CMYK
        else:
            pic0 = fitz.Pixmap(fitz.csRGB, pic)
            pic0.save(os.path.join(pic_path, new_name))
            document.add_picture(os.path.join(pic_path, new_name),width=Inches(5.5))
            pic0 = None
        # 释放资源
        pic = None
        
        

In [6]:
if __name__=='__main__':
    # pdf路径
    # 建议将文献管理软件中的默认pdf命名改为题目名
    path=r"F:\Endnote库\BTF_VOCs.Data\PDF"
    pic_path=r'F:\Endnote库\BTF_VOCs.Data\PIC'
    if os.path.exists(pic_path):
        pass
    else:
        os.mkdir(pic_path)
    document=Document()
    filenames=[filename for filename in os.listdir(path)]
    n=1
    for i in filenames:
        full_path=path+"\\"+ i
        for root, subDirs, files in os.walk(full_path):
            for fn in files:
                if fn.endswith(".pdf"):
                    heading="["+ str(n) +"]"+" "+fn.strip(".pdf")
                    document.add_heading(heading,level=1)
                    pdf_path=full_path+"\\"+fn
                    fin=open(pdf_path,'rb')
                    picpath=pic_path+"\\"+fn.strip(".pdf")
                    if os.path.exists(picpath):
                        pass
                    else:
                        os.mkdir(picpath)
              
                    pdf2pic(pdf_path,picpath,document)
        n=n+1

    document.save("F:\Endnote库\BTF_VOCs.Data\PDF\demo.docx")


FitzDeprecation: 'writePNG' removed from class 'Pixmap' after v1.19 - use 'save'. [__init__.py:170]
